Task 3 - Evaluation for Domain Generalization - PACS Dataset

Making All the required Imports

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import deeplake
import os
from PIL import Image
import torchvision.models as models

Loading In the VGG Disciminative Model and Moving it to the GPU if Available

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.vgg19(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Getting The VGG Pretraind Model Ready for training.

This step includes freezing all parameters in the feature extractor and replacing the classifier head with a new head based on the number of classes our dataset has which is 7. We then define loss function and optimizer as.

In [13]:
for param in model.features.parameters():
    param.requires_grad = False  

num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 7)  

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001) 

Loading In The PACS Dataset

Defining our Custom Dataset Wrapper Class to convert Deep Lake tensors into pytorch tensors

In [14]:
class PACSDataset(Dataset):
    def __init__(self, deeplake_dataset, transform=None):
        self.ds = deeplake_dataset
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        image = self.ds['images'][idx].numpy() 
        label = int(self.ds['labels'][idx].numpy()) 

        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)

        return image, label

Load in PACS DATA set using Deep Lake and define the train and validation dataset

In [15]:
train_dataset = deeplake.load("hub://activeloop/pacs-train")
val_dataset = deeplake.load("hub://activeloop/pacs-val")

-

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-train



-

hub://activeloop/pacs-train loaded successfully.



Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-val



hub://activeloop/pacs-val loaded successfully.



Define Image Transformations. These Image Transformations are passed along with train dataset to our PACSDataset Wrapper. Once Passed A series of transformations are applied to the images, along with converting the deeplake tensors into the corrected workable format that we need.

We also get the trainloader which essentially splits train_dataset into batches or chunks. This can be helpful in making the model train more faster and smoother

In [16]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = PACSDataset(train_dataset, transform=transform)

trainloader = DataLoader(train_dataset, batch_size=4, shuffle=True,num_workers=4)

This portion will now is now finetuning the VGG model. Essentially we will be doing a forward pass, calculating the loss and then updating the weights in the backward pass. This is done for the complete dataset for 3 Epochs. After the model is trained we save it so if we require it later we can easily load it in.


In [17]:
model.train()
num_epochs = 3

for epoch in range(num_epochs):
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad() 

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()  
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))
            pbar.update(1) 

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}], Step [{i + 1}], Loss: {running_loss / (i + 1):.4f}')


save_path = "models/vgg_19_PACS_TASK2.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path) 
print(f'Model saved to {save_path}')


Epoch 1/3:   0%|          | 0/2245 [00:00<?, ?batch/s]<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx

Epoch [1], Step [100], Loss: 1.3932


Epoch 1/3:   9%|▉         | 200/2245 [13:12<2:14:48,  3.96s/batch, loss=1.21]

Epoch [1], Step [200], Loss: 1.2132


Epoch 1/3:  13%|█▎        | 300/2245 [19:45<2:03:44,  3.82s/batch, loss=1.11]

...

Epoch [3], Step [2100], Loss: 0.8790


Epoch 3/3:  98%|█████████▊| 2200/2245 [2:23:57<02:50,  3.80s/batch, loss=0.882]

Epoch [3], Step [2200], Loss: 0.8815


Epoch 3/3: 100%|██████████| 2245/2245 [2:26:53<00:00,  3.93s/batch, loss=0.88]


Model saved to models/vgg_19_PACS_TASK2.pth


Running the Training Loop

Evaluate the VGG model. See how Well the Model performs on an Out of Domain Dataset like PACS. It Represents A covariate Shift so a drop in Accuracy is expected.

In [18]:
val_dataset = PACSDataset(val_dataset, transform=transform)
valloader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(valloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

PACS_Accuracy = 100 * correct / total

print(f'Accuracy on PACS Validation Set: {100 * correct / total:.2f}%')

  0%|          | 0/254 [00:00<?, ?it/s]<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-14-caa64f65a0af>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # C

Accuracy on PACS Validation Set: 78.01%
